In [1]:
import numpy as np
import pandas as pd
import pywt

In [97]:
df = pd.read_csv('Ejemplos/Distribuciones/Binomial-fin.csv',index_col='indice')

In [3]:
def series_periodos(inicio, periodos, freq): 
    serie = pd.date_range(start=inicio, periods=periodos, freq=freq)
    return serie

DESPLAZAMIENTO TEMPORAL Y TRASLACION

In [4]:
def time_shift(series, shift):
    return series.shift(shift)

In [5]:
def traslacion(df,shift):
    for x in df.columns:
        df[x] = df[x] + shift
    return df

In [11]:
df

,Valor
indice,
2014-01-31,53
2014-02-28,52
2014-03-31,51
2014-04-30,52
2014-05-31,50
...,...
2023-08-31,51
2023-09-30,50
2023-10-31,50


In [12]:
traslacion(df,10)

,Valor
indice,
2014-01-31,63
2014-02-28,62
2014-03-31,61
2014-04-30,62
2014-05-31,60
...,...
2023-08-31,61
2023-09-30,60
2023-10-31,60


AGREGAR RUIDO:

In [16]:
def add_noise(series, noise_level=0.1):
    noise = np.random.normal(0, noise_level, len(series))
    return series + noise

In [14]:
def df_ruido(df,noise_level=0.1):
    for x in df.columns:
        df[x] = add_noise(df[x],noise_level)
    return df

In [17]:
df_ruido(df,0.4)

,Valor
indice,
2014-01-31,62.886928
2014-02-28,61.730235
2014-03-31,62.068131
2014-04-30,61.386790
2014-05-31,59.963889
...,...
2023-08-31,60.677755
2023-09-30,59.705584
2023-10-31,59.668497


ESCALADO

In [19]:
def scale(series,factor):
    return series*factor

In [20]:
def df_scale(df,factor):
    for x in df.columns:
        df[x] = scale(df[x],factor)
    return df

In [21]:
df_scale(df,0.45)

,Valor
indice,
2014-01-31,28.299117
2014-02-28,27.778606
2014-03-31,27.930659
2014-04-30,27.624056
2014-05-31,26.983750
...,...
2023-08-31,27.304990
2023-09-30,26.867513
2023-10-31,26.850824


RECORTE

In [22]:
def crop(series, start, end):
    return series[start:end]

In [23]:
def recorte(df, start,end):
    for x in df.columns:
        df[x] = crop(df[x],start,end)
    df=df.dropna()
    return df

In [24]:
recorte(df,15,100)

,Valor
indice,
2015-04-30,27.399621
2015-05-31,27.976985
2015-06-30,26.887157
2015-07-31,27.334492
2015-08-31,27.039215
...,...
2021-12-31,27.476207
2022-01-31,26.878574
2022-02-28,27.575679


INTERPOLACION:

In [29]:
from scipy.interpolate import interp1d
from scipy.interpolate import UnivariateSpline

In [40]:
def interpolacion(df,kind,num,freq):
    df=df.reset_index()
    indices=df.index.values
    indice=series_periodos(df[df.columns[0]][0],num,freq)
    x =indices 
    for i in range(1,len(df.columns)):
        y = df[df.columns[i]]
        f = interp1d(x, y, kind=kind) #kind='linear' / 'cubic'
        x_new = np.linspace(0, 4, num=num)  # New x values
        y_new = f(x_new)  # Interpolated y values
        if i==1:
            df_int = pd.DataFrame(data=y_new,index=indice,columns=[df.columns[i]])
        else :     
            df_n = pd.DataFrame(data=y_new,index=indice,columns=[df.columns[i]])
            df_int= df_int.join(df_n, how="outer")
    return df_int

In [43]:
interpolacion(df,'linear',500,'M')

,Valor
2014-01-31,53.000000
2014-02-28,52.991984
2014-03-31,52.983968
2014-04-30,52.975952
2014-05-31,52.967936
...,...
2055-04-30,50.064128
2055-05-31,50.048096
2055-06-30,50.032064
2055-07-31,50.016032


In [44]:
interpolacion(df,'cubic',140,'M')

,Valor
2014-01-31,53.000000
2014-02-28,53.013821
2014-03-31,53.023910
2014-04-30,53.030379
2014-05-31,53.033338
...,...
2025-04-30,50.239558
2025-05-31,50.176059
2025-06-30,50.114805
2025-07-31,50.056038


In [45]:
def interpolate(data):
    interpolated_data = []
    for i in range(len(data) - 1):
        interpolated_data.append(data[i])
        interpolated_data.append((data[i] + data[i + 1]) / 2)  # Punto intermedio
    interpolated_data.append(data[-1])
    return np.array(interpolated_data)

In [47]:
def punto_medio(df,freq):
    for x in df.columns:
        data = df[x]
        a = interpolate(data)
        if x == df.columns[0]:
            indice = series_periodos(df.index[0],len(a),freq)
            df_pm = pd.DataFrame(data=a,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=a,index=indice,columns=[x])
            df_pm = df_pm.join(df_new, how="outer")
    return df_pm

In [48]:
punto_medio(df,'M')

,Valor
2014-01-31,53.0
2014-02-28,52.5
2014-03-31,52.0
2014-04-30,51.5
2014-05-31,51.0
...,...
2033-07-31,50.0
2033-08-31,50.5
2033-09-30,51.0
2033-10-31,51.0


In [49]:
def spline_interpolation(data, s=1):
    x = np.arange(len(data))
    spline = UnivariateSpline(x, data, s=s)
    return spline(x)

In [50]:
def interpolacion_spline(df,s):
    for x in df.columns:
        df[x]=spline_interpolation(df[x],s)
    return df

In [52]:
interpolacion_spline(df,3)

,Valor
indice,
2014-01-31,53.003479
2014-02-28,51.981536
2014-03-31,51.040921
2014-04-30,51.954335
2014-05-31,50.015175
...,...
2023-08-31,50.775734
2023-09-30,49.982316
2023-10-31,50.186066


RANDOM SAMPLING:

In [54]:
# Randomly sampling with replacement
def sampling(df,size,freq):
    indice=series_periodos(df.index[0],size,freq)
    for x in df.columns:
        data = df[x]
        sampled_data = np.random.choice(data, size=size, replace=True)
        if x == df.columns[0]:
            df_sampling=pd.DataFrame(data=sampled_data,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=sampled_data,index=indice,columns=[x])
            df_sampling= df_sampling.join(df_new, how="outer")
    return df_sampling

In [56]:
sampling(df,200,'M')

,Valor
2014-01-31,50.279883
2014-02-28,51.357314
2014-03-31,50.279883
2014-04-30,50.954019
2014-05-31,50.248084
...,...
2030-04-30,49.894225
2030-05-31,51.942310
2030-06-30,50.521995
2030-07-31,51.003732


TRANSFORMACIÓN MATEMÁTICA:

In [57]:
def agregar_log(df):
    for x in df.columns:
        df[x] = np.log1p(df[x])
    return df 

In [58]:
def agregar_sqrt(df):
    for x in df.columns:
        df[x] = np.sqrt(df[x])
    return df 

In [59]:
def agregar_exp(df,factor):
    for x in df.columns:
        df[x] = np.exp(df[x]/factor)
    return df 

In [60]:
def agregar_sin(df):
    for x in df.columns:
        df[x] = np.sin(df[x])
    return df

In [61]:
def agregar_cos(df):
    for x in df.columns:
        df[x] = np.cos(df[x])
    return df

In [62]:
def agregar_trig(df):
    for x in df.columns:
        df[x] = np.cos(df[x]) + np.sin(df[x])
    return df 

In [63]:
def agregar_sigmoid(df):
    for x in df.columns:
        df[x] = 1 / (1 + np.exp(-df[x]))
    return df
   

MIXUP

In [66]:
def mixup(data, alpha=0.2):
    lambda_ = np.random.beta(alpha, alpha)
    indices = np.random.permutation(len(data))
    data_mixup = lambda_ * data + (1 - lambda_) * data[indices]
    return data_mixup

In [64]:
def agregar_mixup(df,alpha=0.2):
    for x in df.columns:
        df[x] = mixup(df[x].values,alpha)
    return df

In [67]:
agregar_mixup(df)

,Valor
indice,
2014-01-31,49.866847
2014-02-28,51.982777
2014-03-31,51.035182
2014-04-30,50.330684
2014-05-31,51.316595
...,...
2023-08-31,50.967774
2023-09-30,51.008804
2023-10-31,50.015476


In [70]:
def random_mix(data, n_samples=100):
    mixed_data = []
    for _ in range(n_samples):
        i, j = np.random.choice(len(data), 2, replace=False)
        mixed_data.append((data[i] + data[j]) / 2)
    return np.array(mixed_data)

In [69]:
def agregar_random_mix(df,freq,n_samples):
    indice=series_periodos(df.index[0],n_samples,freq)
    for x in df.columns:
        sampled_data = random_mix(df[x],n_samples)
        if x == df.columns[0]:
            df_sampling=pd.DataFrame(data=sampled_data,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=sampled_data,index=indice,columns=[x])
            df_sampling= df_sampling.join(df_new, how="outer")
    return df_sampling

In [71]:
agregar_random_mix(df,'M',380)

,Valor
2014-01-31,50.243961
2014-02-28,50.098233
2014-03-31,50.454804
2014-04-30,49.980509
2014-05-31,50.447474
...,...
2045-04-30,50.552895
2045-05-31,51.868280
2045-06-30,51.436472
2045-07-31,50.691769


VENTANAS DESLIZANTES:

In [81]:
def ventanas(df,ventana):
    for x in df.columns:
        df[x] = df[x].rolling(window=ventana).mean()
        df = df.dropna()
        return df

In [82]:
ventanas(df,4)

,Valor
indice,
2015-07-31,50.935691
2015-08-31,51.032347
2015-09-30,51.087081
2015-10-31,51.087410
2015-11-30,51.046871
...,...
2023-08-31,50.916587
2023-09-30,50.951200
2023-10-31,50.934732


PCA:

In [ ]:
from sklearn.decomposition import PCA

data_multidim = np.array([[1.2, 3.4], [5.6, 7.8], [9.1, 2.2]])
pca = PCA(n_components=2)
data_pca = pca.fit_transform(data_multidim)
data_pca_jittered = data_pca + np.random.normal(0, 0.1, data_pca.shape)
data_augmented = pca.inverse_transform(data_pca_jittered)

USO DE TECNICAS ESTADÍSTICAS:

In [100]:
def normal(df,freq,size):
    indice=series_periodos(df.index[0],size+df.shape[0],freq)
    for x in df.columns:
        data = df[x]
        mean,std_dev = np.mean(data),np.std(data)
        data_augmented = np.random.normal(mean,std_dev,size=size)
        if x == df.columns[0]:
            df_normal=pd.DataFrame(data=np.concatenate((df[x].values,data_augmented)),index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=np.concatenate((data,data_augmented)),index=indice,columns=[x])
            df_normal= df_normal.join(df_new, how="outer")
    return df_normal

In [101]:
normal(df,'M',300)

,Valor
2014-01-31,53.000000
2014-02-28,52.000000
2014-03-31,51.000000
2014-04-30,52.000000
2014-05-31,50.000000
...,...
2048-08-31,51.259481
2048-09-30,50.391287
2048-10-31,50.858691
2048-11-30,51.570037


In [105]:
def log_normal(df,freq,sigma,size):
    indice=series_periodos(df.index[0],size+df.shape[0],freq)
    for x in df.columns:
        data = df[x].values
        data_augmented = np.random.lognormal(mean=np.log(data.mean()),sigma=sigma,size=size)
        if x == df.columns[0]:
            df_lognormal=pd.DataFrame(data=np.concatenate((df[x].values,data_augmented)),index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=np.concatenate((data,data_augmented)),index=indice,columns=[x])
            df_lognormal= df_lognormal.join(df_new, how="outer")
    return df_lognormal

In [106]:
log_normal(df,'M',0.5,200)

,Valor
2014-01-31,53.000000
2014-02-28,52.000000
2014-03-31,51.000000
2014-04-30,52.000000
2014-05-31,50.000000
...,...
2040-04-30,55.813829
2040-05-31,67.479704
2040-06-30,145.628254
2040-07-31,88.872659


In [110]:
def box_muller_transform(mean, std_dev, size=100):
    u1, u2 = np.random.rand(size), np.random.rand(size)
    z1 = np.sqrt(-2 * np.log(u1)) * np.cos(2 * np.pi * u2)
    return mean + z1 * std_dev

def box_muller(df,freq,size):
    indice=series_periodos(df.index[0],size+df.shape[0],freq)
    for x in df.columns:
        data = df[x].values
        data_bm = box_muller_transform(data.mean(),data.std(),size)
        if x == df.columns[0]:
            df_bm=pd.DataFrame(data=np.concatenate((df[x].values,data_bm)),index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data=np.concatenate((df[x].values,data_bm)),index=indice,columns=[x])
            df_bm = df_bm.join(df_new, how="outer")
    return df_bm

In [111]:
box_muller(df,'M',300)

,Valor
2014-01-31,53.000000
2014-02-28,52.000000
2014-03-31,51.000000
2014-04-30,52.000000
2014-05-31,50.000000
...,...
2048-08-31,51.138977
2048-09-30,51.547536
2048-10-31,51.450927
2048-11-30,49.517649


DESPLAZAMIENTO EN BLOQUES:

In [118]:
def despl_bloque(df,block_size):
    for x in df.columns:
        data = df[x].values
        shifted_data = []
        for i in range(0,len(data),block_size):
            block = data[i:i+block_size]
            shifted_block = np.roll(block, np.random.randint(1, block_size))
            shifted_data.extend(shifted_block)
        df[x] = np.array(shifted_data)
    return df

In [114]:
df

,Valor
indice,
2014-01-31,50
2014-02-28,53
2014-03-31,52
2014-04-30,51
2014-05-31,52
...,...
2023-08-31,51
2023-09-30,51
2023-10-31,51


In [119]:
despl_bloque(df,5)

,Valor
indice,
2014-01-31,52
2014-02-28,51
2014-03-31,52
2014-04-30,50
2014-05-31,53
...,...
2023-08-31,50
2023-09-30,50
2023-10-31,51


ROTACIÓN:

In [120]:
def rotacion(df):
    for x in df.columns:
        data = df[x].values
        rotation_steps = np.random.randint(1, len(data))
        df[x]= np.roll(data, rotation_steps)
    return df

In [121]:
rotacion(df)

,Valor
indice,
2014-01-31,50
2014-02-28,51
2014-03-31,50
2014-04-30,50
2014-05-31,53
...,...
2023-08-31,51
2023-09-30,50
2023-10-31,50


RUIDO ARMONICO

In [131]:
def add_harmonic_noise(df, amplitude=0.1, frequency=0.5):
    for x in df.columns:
        data = df[x].values
        time = np.arange(len(data))
        harmonic_noise = amplitude * np.sin(2 * np.pi * frequency * time)
        df[x] = data + harmonic_noise
    return df

In [133]:
add_harmonic_noise(df,1000,0.5)

,Valor
indice,
2014-01-31,50.0
2014-02-28,51.0
2014-03-31,50.0
2014-04-30,50.0
2014-05-31,53.0
...,...
2023-08-31,51.0
2023-09-30,50.0
2023-10-31,50.0


BOOTSTRAPPING:

In [134]:
def agregar_bootstrapping(df,freq):
    # Suponiendo  que df es tu DataFrame con series temporales originales
    
    for x in df.columns:
        synthetic_data = df.sample(frac=1, replace=True).reset_index(drop=True)
        synthetic_data[x] += np.random.normal(0, 0.1, len(synthetic_data))  # Añadir ruido
        indice=series_periodos(df.index[0],len(df)+len(synthetic_data),freq)
        a=pd.concat([df[x],synthetic_data[x]])
        a.index=indice
        if x == df.columns[0]:
            df_bootstrap=a
        else:
            df_bootstrap= df_bootstrap.join(a, how="outer")
    return df_bootstrap

In [135]:
agregar_bootstrapping(df,'M')

2014-01-31    50.000000
2014-02-28    51.000000
2014-03-31    50.000000
2014-04-30    50.000000
2014-05-31    53.000000
                ...    
2033-08-31    50.901690
2033-09-30    50.105336
2033-10-31    49.970351
2033-11-30    50.154803
2033-12-31    49.891427
Freq: M, Name: Valor, Length: 240, dtype: float64

PERTURBACIÓN GAUSSIANA:

In [138]:
def cumulative_gaussian_perturbation(data, std_dev=0.05):
    perturbations = np.random.normal(0, std_dev, len(data))
    cumulative_data = np.cumsum(perturbations) + data
    return cumulative_data

In [136]:
def agregacion_gauss(df,std_dev=0.05):
    for x in df.columns:
        df[x] = cumulative_gaussian_perturbation(df[x],std_dev)
    return df

In [139]:
agregacion_gauss(df,0.1)

,Valor
indice,
2014-01-31,50.077089
2014-02-28,51.140813
2014-03-31,49.945779
2014-04-30,49.963889
2014-05-31,52.941948
...,...
2023-08-31,51.655694
2023-09-30,50.799683
2023-10-31,50.767895


TRANSFORMACIÓN WAVELET:

In [142]:
def wavelet_transform_augmentation(data, wavelet='db1'):
    coeffs = pywt.wavedec(data, wavelet)
    coeffs[0] += np.random.normal(0, 0.1, len(coeffs[0]))  # Añadir ruido al coeficiente principal
    augmented_data = pywt.waverec(coeffs, wavelet)
    return augmented_data[:len(data)]  # Para mantener el tamaño original


In [141]:
def agregar_wavelet(df,wavelet='db1'):
    for x in df.columns:
        data = df[x].values
        df[x]=wavelet_transform_augmentation(data,wavelet)
    return df

In [143]:
agregar_wavelet(df)

,Valor
indice,
2014-01-31,50.091752
2014-02-28,51.155476
2014-03-31,49.960442
2014-04-30,49.978552
2014-05-31,52.956611
...,...
2023-08-31,51.645029
2023-09-30,50.789018
2023-10-31,50.757230


DUPLICADO + PERTURBACIÓN:

In [144]:
def duplicate_and_perturb(data, duplication_factor=0.3, perturbation_std=0.05):
    duplicated_data = []
    np.random.seed(5)
    for point in data:
        duplicated_data.append(point)
        if np.random.rand() < duplication_factor:
            duplicated_data.append(point + np.random.normal(0, perturbation_std))
    return np.array(duplicated_data)

In [145]:
def duplicados(df,freq,duplication_factor=0.3,perturbation_std=0.05):
    
    for x in df.columns:
        data = df[x]
        data_dd=duplicate_and_perturb(data,duplication_factor,perturbation_std)
        if x == df.columns[0]:
            indice = series_periodos(df.index[0],len(data_dd),freq)
            df_dd = pd.DataFrame(data=data_dd,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data = data_dd,index=indice,columns=[x])
            df_dd = df_dd.join(df_new, how="outer")
    return df_dd

In [146]:
duplicados(df,'M')

,Valor
2014-01-31,50.091752
2014-02-28,50.077054
2014-03-31,51.155476
2014-04-30,49.960442
2014-05-31,49.978552
...,...
2026-05-31,50.789018
2026-06-30,50.682478
2026-07-31,50.757230
2026-08-31,51.830280


SIMULACIÓN DE SALTOS:

In [147]:
def pulse_noise(data, num_pulses=5, amplitude=1):
    pulse_indices = np.random.choice(len(data), num_pulses, replace=False)
    pulse_data = data.copy()
    for i in pulse_indices:
        pulse_data[i] += np.random.uniform(-amplitude, amplitude)
    return pulse_data

In [151]:
def agregar_saltos(df,num_pulses,amplitude=1):
    for x in df.columns:
        data = df [x]
        data_salto=pulse_noise(data,num_pulses,amplitude)
        df[x]= data_salto
    return df

In [153]:
agregar_saltos(df,30)

,Valor
indice,
2014-01-31,50.091752
2014-02-28,51.155476
2014-03-31,50.949094
2014-04-30,49.978552
2014-05-31,53.782661
...,...
2023-08-31,51.645029
2023-09-30,50.789018
2023-10-31,50.757230


COMBINACIÓN LINEAL:

In [156]:
def linear_combinations(data, n_combinations):
    combinations = []
    for _ in range(n_combinations):
        weights = np.random.rand(data.shape[0])
        weights /= np.sum(weights)  # Normalizar pesos
        combination = np.dot(weights, data)
        combinations.append(combination)
    return np.array(combinations)

In [158]:
def agregar_comb(df,freq,size):
    for x in df.columns:
        data = df[x]
        data_augmented = linear_combinations(data,size)
        datos=np.concatenate((data.values,data_augmented))
        if x == df.columns[0]:
            indice = series_periodos(df.index[0],len(datos),freq)
            df_dl = pd.DataFrame(data=datos,index=indice,columns=[x])
        else:
            df_new = pd.DataFrame(data = datos,index=indice,columns=[x])
            df_dl = df_dl.join(df_new, how="outer")
    return df_dl

In [159]:
agregar_comb(df,'M',1000)

,Valor
2014-01-31,50.091752
2014-02-28,51.155476
2014-03-31,50.949094
2014-04-30,49.978552
2014-05-31,53.782661
...,...
2106-12-31,50.946805
2107-01-31,51.088001
2107-02-28,51.026965
2107-03-31,50.939780
